In [2]:
import pickle
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
INTERSECTION = 0
BRIDGE = 1

In [12]:
# Load road network
G = nx.read_gpickle("quick_traffic_model/input/graphMTC_CentroidsLength3int.gpickle")

# Make intersections node type 0 to separate from bridges
for n in G.nodes():
    G.node[n]['nodetype'] = INTERSECTION # intersection, not bridge

print 'finished loading road network'
G = G.to_undirected()

# Open bridge database
bridges_dict = None
with open('quick_traffic_model/input/20140114_master_bridge_dict.pkl','rb') as f:
    bridges_dict = pickle.load(f)

# Add list of bridges on each road segment
for b_num in bridges_dict.keys():
    bridge_edges = bridges_dict[b_num]['a_b_pairs_direct']
    new_id = str(bridges_dict[b_num]['new_id'])
    for [u,v] in bridge_edges:
        G.edge[str(u)][str(v)]['bridges'].append('b' + new_id)
        
intersections_G = G.copy()

# For each bridge, make a new node connecting to the proper intersections
# in series, needs to be changed to in parallel for modularity to work properly
for (u,v) in G.edges():
    if len(G.edge[u][v]['bridges']) > 0:
        bridge_list = G.edge[u][v]['bridges']
        for bridge in bridge_list:
            data = G.get_edge_data(u, v)
            data['bridges'] = None
            data['distance'] = data['distance']/2
            data['distance_0'] = data['distance_0']/2
            data['t_0'] = data['t_0']/2
            G.add_node(bridge, nodetype=BRIDGE)
            G.add_edge(u, bridge, attr_dict = data)
            G.add_edge(bridge, v, attr_dict = data)
        G.remove_edge(u,v)

print 'finished adding bridges'
bridges_and_intersections_G = G.copy()

# Remove the intersections
for n in G.nodes():
    if G.node[n]['nodetype'] == INTERSECTION:
        adj_nodes = G.neighbors(n)
        for v1 in range(len(adj_nodes)):
            for v2 in range(v1+1, len(adj_nodes)):
                v1n = adj_nodes[v1]
                v2n = adj_nodes[v2]
                if G.node[v1n]['nodetype'] == BRIDGE and G.node[v2n]['nodetype'] == BRIDGE:
                    G.add_edge(v1n, v2n)
        G.remove_node(n)

print 'finished removing road intersections'
bridges_G = G.copy()

finished loading road network
finished adding bridges
finished removing road intersections


In [16]:
#print G.nodes()
#print G.edges()
print 'INTERSECTIONS'
for (u,v) in intersections_G.edges():
    if len(intersections_G.get_edge_data(u, v)['bridges']) > 1:
        print u,v,intersections_G.get_edge_data(u, v)
    
# 7016 6970 {'distance': 2.079999, 'dailyvolume': 133477, 'capacity_0': 9500, 'lanes': 5, 
# 'bridges': ['b956', 'b960', 'b1743'], 't_a': 124, 'flow': 0, 'distance_0': 2.079999, 'capacity': 9500, 't_0': 124}

    
#print 'INTERSECTIONS AND BRIDGES'
#for (u,v) in intersections_G.edges()[:100]:
#    print u,v,bridges_and_intersections_G.get_edge_data(u, v)

INTERSECTIONS
11549 2202 {'distance': 2.839999, 'dailyvolume': 23628, 'capacity_0': 4000, 'lanes': 2, 'bridges': ['b1239', 'b1247'], 't_a': 157, 'flow': 0, 'distance_0': 2.839999, 'capacity': 4000, 't_0': 157}
5983 5885 {'distance': 0.67, 'dailyvolume': 26970, 'capacity_0': 2850, 'lanes': 3, 'bridges': ['b444', 'b444'], 't_a': 80, 'flow': 0, 'distance_0': 0.67, 'capacity': 2850, 't_0': 80}
6797 6155 {'distance': 0.43, 'dailyvolume': 0, 'capacity_0': 650, 'lanes': 1, 'bridges': ['b740', 'b740'], 't_a': 77, 'flow': 0, 'distance_0': 0.43, 'capacity': 650, 't_0': 77}
4641 4646 {'distance': 2.269999, 'dailyvolume': 36585, 'capacity_0': 5850, 'lanes': 3, 'bridges': ['b360', 'b308', 'b315'], 't_a': 125, 'flow': 0, 'distance_0': 2.269999, 'capacity': 5850, 't_0': 125}
2146 2150 {'distance': 1.139999, 'dailyvolume': 352, 'capacity_0': 950, 'lanes': 1, 'bridges': ['b1154', 'b1154'], 't_a': 117, 'flow': 0, 'distance_0': 1.139999, 'capacity': 950, 't_0': 117}
4645 5570 {'distance': 2.269999, 'dail

8279 8283 {'distance': 6.599999, 'dailyvolume': 6081, 'capacity_0': 950, 'lanes': 1, 'bridges': ['b1438', 'b1438'], 't_a': 593, 'flow': 0, 'distance_0': 6.599999, 'capacity': 950, 't_0': 593}
8270 8269 {'distance': 0.829999, 'dailyvolume': 33528, 'capacity_0': 3900, 'lanes': 2, 'bridges': ['b1546', 'b1538'], 't_a': 45, 'flow': 0, 'distance_0': 0.829999, 'capacity': 3900, 't_0': 45}
8270 8591 {'distance': 1.6, 'dailyvolume': 46601, 'capacity_0': 3900, 'lanes': 2, 'bridges': ['b1535', 'b1527'], 't_a': 88, 'flow': 0, 'distance_0': 1.6, 'capacity': 3900, 't_0': 88}
8419 8046 {'distance': 1.6, 'dailyvolume': 40233, 'capacity_0': 3900, 'lanes': 2, 'bridges': ['b1562', 'b1657'], 't_a': 88, 'flow': 0, 'distance_0': 1.6, 'capacity': 3900, 't_0': 88}
8410 8247 {'distance': 1.6, 'dailyvolume': 42043, 'capacity_0': 5700, 'lanes': 3, 'bridges': ['b1588', 'b1459'], 't_a': 96, 'flow': 0, 'distance_0': 1.6, 'capacity': 5700, 't_0': 96}
6663 6691 {'distance': 0.17, 'dailyvolume': 9603, 'capacity_0': 65

6930 6859 {'distance': 0.34, 'dailyvolume': 76434, 'capacity_0': 7800, 'lanes': 4, 'bridges': ['b863', 'b892'], 't_a': 20, 'flow': 0, 'distance_0': 0.34, 'capacity': 7800, 't_0': 20}
6931 6839 {'distance': 0.178929, 'dailyvolume': 9598, 'capacity_0': 2700, 'lanes': 3, 'bridges': ['b888', 'b888'], 't_a': 21, 'flow': 0, 'distance_0': 0.178929, 'capacity': 2700, 't_0': 21}
8699 8606 {'distance': 1.7, 'dailyvolume': 1598, 'capacity_0': 1700, 'lanes': 2, 'bridges': ['b1717', 'b1717'], 't_a': 174, 'flow': 0, 'distance_0': 1.7, 'capacity': 1700, 't_0': 174}
8348 8797 {'distance': 1.0, 'dailyvolume': 1661, 'capacity_0': 950, 'lanes': 1, 'bridges': ['b1662', 'b1662'], 't_a': 102, 'flow': 0, 'distance_0': 1.0, 'capacity': 950, 't_0': 102}
4075 8769 {'distance': 0.3, 'dailyvolume': 619, 'capacity_0': 950, 'lanes': 1, 'bridges': ['b947', 'b947'], 't_a': 30, 'flow': 0, 'distance_0': 0.3, 'capacity': 950, 't_0': 30}
3962 4014 {'distance': 0.3, 'dailyvolume': 18684, 'capacity_0': 1960, 'lanes': 2, 'b

In [22]:
# all on same road segment
print 'b956', 'b960', bridges_G.get_edge_data('b956', 'b960')
print 'b1743', 'b960', bridges_G.get_edge_data('b1743', 'b960')
print 'b956', 'b1743', bridges_G.get_edge_data('b956', 'b1743')

# b959 is adjacent to all 3 if in parallel, not if in series
print 'b959', 'b956', bridges_G.get_edge_data('b956', 'b960')
print 'b959', 'b960', bridges_G.get_edge_data('b1743', 'b960')
print 'b959', 'b1743', bridges_G.get_edge_data('b956', 'b1743')

# confirm that gives different results than missing edges
print 'fake', 'fake2', bridges_G.get_edge_data('fake', 'fake2')

b956 b960 {}
b1743 b960 {}
b956 b1743 {}
b959 b956 {}
b959 b960 {}
b959 b1743 {}
fake fake2 None


In [6]:
print(intersections_G.nodes())

['3480', '11542', '11543', '11540', '11546', '11547', '11544', '11545', '11548', '11549', '5988', '5989', '10610', '5982', '5983', '5980', '5981', '5986', '5987', '5984', '5985', '6796', '6797', '6794', '6795', '6792', '6793', '6790', '6791', '6798', '6799', '4641', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '12019', '12018', '2146', '12017', '12016', '12010', '12013', '3486', '10709', '9528', '9529', '9258', '9259', '9524', '9257', '9526', '9255', '9520', '9521', '9522', '9523', '1370', '5257', '1373', '4645', '1372', '3519', '3518', '3513', '3512', '3511', '3510', '3517', '3516', '3515', '3514', '2688', '2689', '2684', '2685', '2686', '2687', '2680', '2681', '2682', '2683', '99', '98', '91', '90', '93', '92', '95', '94', '97', '96', '8243', '4826', '4524', '1177', '1176', '1175', '1174', '1173', '1172', '1171', '1170', '1179', '1178', '2025', '7188', '7189', '3431', '7180', '7181', '7182', '7183', '7184', '3432', '7186', '7187', '1227', '3433', '1226', '343